In [93]:
import pandas  as pd

import matplotlib.pyplot as plt

train = pd.read_csv('Dataset/Train.csv')
test = pd.read_csv('Dataset/Test.csv')

train['Date'] = pd.to_datetime(train.Date, format = '%Y-%m-%d')
train.index=train['Date']

test['Date'] = pd.to_datetime(test.Date, format = '%Y-%m-%d')
test.index=test['Date']


    


In [94]:
import numpy as np
df=train

In [95]:
#importing required libraries
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM

#creating dataframe

df['Date'] = pd.to_datetime(df.Date,format='%Y-%m-%d')
df.index = df['Date']


data = df.sort_index(ascending=True, axis=0)
new_data = pd.DataFrame(index=range(0,len(df)),columns=['Date', 'Close'])
for i in range(0,len(data)):
    new_data['Date'][i] = data['Date'][i]
    new_data['Close'][i] = data['Close'][i]

#setting index
new_data.index = new_data.Date
new_data.drop('Date', axis=1, inplace=True)

#creating train and test sets
dataset = new_data.values

train = dataset[0:39000,:]
valid = dataset[39000:,:]

#converting dataset into x_train and y_train
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)


In [101]:
scaled_data

array([[0.24876391],
       [0.21899464],
       [0.17998901],
       ...,
       [0.31812251],
       [0.40698393],
       [0.13466557]])

In [102]:
x_train, y_train = [], []
for i in range(60,len(train)):
    x_train.append(scaled_data[i-60:i,0])
    y_train.append(scaled_data[i,0])
x_train, y_train = np.array(x_train), np.array(y_train)

x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))




In [103]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1],1)))
model.add(LSTM(units=50))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x_train, y_train, epochs=10, batch_size=1, verbose=2)

#predicting 246 values, using past 60 from the train data
inputs = new_data[len(new_data) - len(valid) - 60:].values
inputs = inputs.reshape(-1,1)
inputs  = scaler.transform(inputs)

X_test = []
for i in range(60,inputs.shape[0]):
    X_test.append(inputs[i-60:i,0])
X_test = np.array(X_test)

X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))
closing_price = model.predict(X_test)
closing_price = scaler.inverse_transform(closing_price)

Epoch 1/10


KeyboardInterrupt: 

In [42]:
# for prediction on test data
#creating dataframe
df_test= test
df_test['Date'] = pd.to_datetime(df_test.Date,format='%Y-%m-%d')
df_test.index = df_test['Date']

In [43]:
df_test.values

array([['2011-01-01T00:00:00.000000000'],
       ['2011-01-02T00:00:00.000000000'],
       ['2011-01-03T00:00:00.000000000'],
       ...,
       ['2017-12-27T00:00:00.000000000'],
       ['2017-12-28T00:00:00.000000000'],
       ['2017-12-29T00:00:00.000000000']], dtype='datetime64[ns]')

In [44]:
inputs = df_test.values
inputs = inputs.reshape(-1,1)
# inputs  = scaler.transform(inputs)

X_test = np.array(inputs)

X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))
closing_price = model.predict(X_test)
closing_price = scaler.inverse_transform(closing_price)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported numpy type: NPY_DATETIME).

In [40]:
X_test,df_test

(array([[[4.44252163e+15]],
 
        [[4.44281829e+15]],
 
        [[4.44311496e+15]],
 
        ...,
 
        [[5.19960445e+15]],
 
        [[5.19990111e+15]],
 
        [[5.20019778e+15]]]),
                  Date
 Date                 
 2011-01-01 2011-01-01
 2011-01-02 2011-01-02
 2011-01-03 2011-01-03
 2011-01-04 2011-01-04
 2011-01-05 2011-01-05
 ...               ...
 2017-12-25 2017-12-25
 2017-12-26 2017-12-26
 2017-12-27 2017-12-27
 2017-12-28 2017-12-28
 2017-12-29 2017-12-29
 
 [2555 rows x 1 columns])